In [1]:
import polars as pl
import duckdb
import utils_database as ud
import utils
import utils_model as um
%load_ext autoreload
%autoreload 2

In [8]:
# ud.start_session(user_id = 5,material_id=16)

In [3]:
# ud.clear_sessions()
# ud.clear_messages()
# len(ud.list_sessions(user_id=4))

In [2]:
df_suppliers = pl.read_excel(source="data/master_data.xlsx",sheet_name='Users')
df_materials = pl.read_excel(source="data/master_data.xlsx",sheet_name='Materials')
df_contracts = pl.read_excel(source="data/master_data.xlsx",sheet_name='Contracts')
df_sessions = pl.read_excel(source="data/master_data.xlsx",sheet_name='Sessions')

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 10, falling back to string


In [3]:
df_materials.tail()

material_id,description,quantity,price_per_unit,min_quantity,max_quantity,min_price_per_unit,max_price_per_unit,min_incoterm,max_incoterm,min_delivery_time,max_delivery_time,min_payment_term,max_payment_term
i64,str,i64,i64,i64,f64,i64,f64,str,str,i64,i64,str,str
13,"""Material 13""",15436,103,15436,20066.8,103,139.05,"""DDP""","""EXW""",7,28,"""NET10""","""NET90"""
14,"""Material 14""",10057,134,10057,13074.1,134,180.9,"""DDP""","""EXW""",7,28,"""NET10""","""NET90"""
15,"""Material 15""",16001,122,16001,20801.3,122,164.7,"""DDP""","""EXW""",7,28,"""NET10""","""NET90"""
16,"""Material 16""",17589,100,17589,22865.7,100,135.0,"""DDP""","""EXW""",7,28,"""NET10""","""NET90"""
17,"""Material 17""",16048,134,16048,20862.4,134,180.9,"""DDP""","""EXW""",7,28,"""NET10""","""NET90"""


In [4]:
# df_contracts = df_contracts.with_columns(
#     pl.struct([
#         'price_per_unit',
#         'quantity',
#         'bundling_unit',
#         'bundling_amount',
#         'bundling_discount',
#         'payment_term',
#         'payment_term_markup',
#         'delivery_timeline',
#         'contract_period',
#         'contract_inflation',
#         'rebates_threshold_unit',
#         'rebates_discount',
#         'warranty',
#         'incoterms'
#     ])
#     .map_elements(
#         lambda row: um.levers_to_offer_text(
#             price_per_unit = row['price_per_unit'],
#             quantity = row['quantity'],
#             bundling_unit = row['bundling_unit'],
#             bundling_amount = row['bundling_amount'],
#             bundling_discount = row['bundling_discount'],
#             payment_term = row['payment_term'],
#             payment_term_markup = row['payment_term_markup'],
#             delivery_timeline = row['delivery_timeline'],
#             contract_period = row['contract_period'],
#             contract_inflation = row['contract_inflation'],
#             rebates_threshold_unit = row['rebates_threshold_unit'],
#             rebates_discount = row['rebates_discount'],
#             warranty = row['warranty'],
#             incoterms = row['incoterms']
#         )
#     )
#     .alias('contract_details')
# )
# df_contracts.head()

In [22]:
df = df_materials.head(2).transpose(include_header=True)
df.columns = ['Levers','Offer 1','Offer 2']
df
# .to_pandas().to_html(index = False)

Levers,Offer 1,Offer 2
str,str,str
"""material_id""","""1""","""2"""
"""description""","""Material 1""","""Material 2"""
"""quantity""","""14548""","""10144"""
"""price_per_unit""","""149""","""104"""
"""min_quantity""","""14548""","""10144"""
…,…,…
"""max_incoterm""","""EXW""","""EXW"""
"""min_delivery_time""","""7""","""7"""
"""max_delivery_time""","""28""","""28"""


In [4]:
df_materials = pl.read_excel(source="data/master_data.xlsx",sheet_name='Materials')
for material in df_materials.rows(named = True):
    material = ud.Material().load_from_data(material)
    ud.add_material(material=material)

In [5]:
df_suppliers = pl.read_excel(source="data/master_data.xlsx",sheet_name='Users')
for user in df_suppliers.rows(named = True):
    ud.add_user(username=user['username'],role=user['role'])

In [6]:
df_contracts = pl.read_excel(source="data/master_data.xlsx",sheet_name='Contracts')
for contract in df_contracts.rows(named = True):
    contract = ud.Contract().load_from_data(contract)
    ud.add_contract(contract=contract)

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 10, falling back to string


In [7]:
# ud.clear_sessions()
df_sessions = pl.read_excel(source="data/master_data.xlsx",sheet_name='Sessions')
for session in df_sessions.rows(named = True):
    session = ud.Session().load_from_data(session)
    ud.add_session(session=session)

In [9]:
con = duckdb.connect("chatbot.duckdb")
df = con.execute("""SELECT * FROM Materials""").pl()
con.close()
df.head(2)

material_id,description,quantity,price_per_unit,min_quantity,max_quantity,min_price_per_unit,max_price_per_unit,min_incoterm,max_incoterm,min_payment_term,max_payment_term,min_delivery_time,max_delivery_time,created_at
i32,str,i32,f32,i32,i32,f32,f32,str,str,str,str,i32,i32,datetime[μs]
1,"""Material 1""",14548,149.0,14548,18912,149.0,201.149994,"""DDP""","""DDP""","""NET10""","""NET90""",7,28,2025-03-13 09:57:08.880
2,"""Material 2""",10144,104.0,10144,13187,104.0,140.399994,"""DDP""","""DDP""","""NET10""","""NET90""",7,28,2025-03-13 09:57:08.926


In [10]:
con = duckdb.connect("chatbot.duckdb")
df = con.execute("""SELECT ss.session_id, u.username,m.description, ss.start_time, ss.end_time 
        FROM Sessions ss
        LEFT JOIN Users u on ss.user_id = u.user_id
        LEFT JOIN Materials m on ss.material_id = m.material_id
        order by ss.start_time desc""").pl()
con.close()
df.head(2)

session_id,username,description,start_time,end_time
i32,str,str,datetime[μs],datetime[μs]
4,"""abc123""","""Material 1""",2025-03-13 00:00:00,2025-03-13 00:00:00
1,"""abc123""","""Material 2""",2025-03-13 00:00:00,2025-03-13 00:00:00


In [11]:
ud.pull_sessions_by_user('monojit')

session_id,username,description,start_time,end_time
i32,str,str,datetime[μs],datetime[μs]
11,"""monojit""","""Material 14""",2025-03-13 00:00:00,2025-03-13 00:00:00
12,"""monojit""","""Material 15""",2025-03-13 00:00:00,2025-03-13 00:00:00
13,"""monojit""","""Material 16""",2025-03-13 00:00:00,2025-03-13 00:00:00


In [14]:
material = ud.get_material_info(material_id = 2)
contract = ud.get_last_contract(user_id=1,material_id=2)
MIN_LEVERS = {
    'price_per_unit':material['min_price_per_unit'],
    'quantity': material['min_quantity']
}
MAX_LEVERS = {
    'price_per_unit':material['max_price_per_unit'],
    'quantity': material['max_quantity']
}
contract

{'contract_id': 2,
 'user_id': 1,
 'material_id': 2,
 'contract_details': None,
 'price_per_unit': 104.0,
 'quantity': 10144.0,
 'bundling_unit': 20288.0,
 'bundling_amount': None,
 'bundling_discount': 5.0,
 'payment_term': 'NET60',
 'delivery_timeline': 17.0,
 'contract_period': 2.0,
 'contract_inflation': 1.0,
 'rebates_threshold_unit': 13085.0,
 'rebates_discount': 2.0,
 'warranty': 1.0,
 'incoterms': 'EXW',
 'created_at': datetime.datetime(2025, 3, 1, 10, 6, 11, 296000),
 'expiry_tmstp': datetime.datetime(2025, 3, 1, 10, 6, 11, 296000)}

In [ ]:
conn = duckdb.connect('chatbot.duckdb')
matrls = ud.list_materials(user_id=1,db_connection=conn)
conn.close()

[(4,), (1,), (3,), (2,), (5,)]